# Superdroplet Breakup: CPU Demo

Created by Emily de Jong, 20 May 2021

## 0. Install instructions

Be sure that an up-to-date version of Python3 is installed in order to build and run this package. Download the source code from: https://github.com/edejong-caltech/PySDM/tree/edj-Breakup. Suggested steps below.

## 1. Background

PySDM is an existing superdroplet-method code that can handle many sets of particle processes, excluding breakup. The purpose of this project is to add a breakup-type process to the codebase for future research and testing. I extend the current implementation of particle coalescence to include breakup as well, with the following algorithm (my contributions and changes marked in green).

1. Randomly order the existing list of superdroplets, and generate a random number for collision probability.

2. Group the ordered list of superdroplets into pairs.

3. a) Compute the probability of a collision, based on collision kernel
      - <span style='color:green'> In the new version, coalescence and breakup take an additional argument: coalescence efficiency $E_c$. This required a minor modification to the backend and front end for "PySDM/dynamics/coalescence.py" and a new module "PySDM/physics/coalescence_efficiences.py". </span>
      - As in the literature, coalescence proceeds with rate $K(x,y)E_c$ and breakup with rate $K(x,y)(1-E_c)$ where $K(x,y)$ is the rate of collisions between particles of size $x$ and $y$.

   b) <span style='color:green'> Compute the number of fragments resulting from collisional breakup. Included are two example fragmentation functions: "PySDM/physics/breakup_fragmentations/slams.py" and ".../alwaysn.py". </span>
   
4. Compute $\gamma$, the number of collisions that lead to <span style='color:green'> breakup </span> (rather than to coalescence).

5. Perform the breakup step.
   

#### <span style='color:green'> Breakup step </span>  

The breakup step that updates particle attributes, which is called from "dynamics/breakup.py" and "state/particles.py", is based on the backend kernel for coalescence with modifications to include the fragmentation. The CPU backend source code is found in <span style='color:green'> "backends/numba/impl/_algorithmic_methods.py" </span>. The steps are as follows:

- If $\gamma = 0$, no collision; continue
- Else, for particles $j, k$ with multiplicities $n[j]$ and $n[k]$ and extrinsic attributes $a[j]$ and $a[k]$, find the new number of particles $n'[j] = n[j] - \gamma n[k]$

- If $n'[j] > 0$, update the attributes and particle counts:
    * $n[j]$ = $n'[j]$, $a[j]$ unchanged
    * $n[k]$ = $n[k] * n_{fragment}$
    * $a[k]$ = $\frac{a[k] + \gamma a[j]}{n_{fragment}}$
- Else split the resulting single droplet $a[k]$ into two superdroplets with equal attributes and half the multiplicity.

- If any SD multiplicities are zero, flag them.